## Install Dependencies

In [1]:
# Install required libraries
!pip install pandas scikit-learn nltk kaggle


     -------------------------------------- 82.7/82.7 kB 421.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 166.4/166.4 kB 624.6 kB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105789 sha256=7a5ea3e5b9eb96ab3a56901f69fb9b5720b77ba4fd032eb8e49da2a7807a5ab7
  Stored in directory: c:\users\muyan\appdata\local\pip\cache\wheels\0b\74\1d\150587ee43d443adecd6ab02f22d569778bdd303377907a9ac
Successfully built kaggle
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.12.7
    Uninstalling certifi-2022.12.7:
      Successfully uninstalled certifi-2022.12.7


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.


In [2]:
import pandas as pd
import os
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle

In [7]:
!pip install kagglehub

     -------------------------------------- 55.2/55.2 kB 408.3 kB/s eta 0:00:00


In [8]:
import kagglehub
import os
import pandas as pd

# Download the dataset
path = kagglehub.dataset_download("ahsanaseer/top-rated-tmdb-movies-10k")

# List the files in the downloaded directory
files = os.listdir(path)
print("Files in dataset:", files)

# Load the dataset (assuming it's a CSV file)
csv_file = [f for f in files if f.endswith('.csv')][0]  # Get the first CSV file
movies = pd.read_csv(os.path.join(path, csv_file))

# Display the first few rows of the dataset
movies.head()


100%|██████████████████████████████████████████████████████████████████████████████| 1.43M/1.43M [00:03<00:00, 482kB/s]

Extracting files...


Files in dataset: ['top10K-TMDB-movies.csv']


,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


## Data Inspection and Feature Selection

In [9]:
# Data exploration
movies.describe()
movies.info()
movies.isnull().sum()

# Feature selection
movies = movies[['id', 'title', 'overview', 'genre']]
movies.head(5)

# Create a 'tags' column
movies['tags'] = movies['overview'] + movies['genre']
movies.head(5)

# Drop 'overview' and 'genre' columns
new_data = movies.drop(columns=['overview', 'genre'])
new_data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


,id,title,tags
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o..."
3,424,Schindler's List,The true story of how businessman Oskar Schind...
4,240,The Godfather: Part II,In the continuing saga of the Corleone crime f...


## Text Cleaning (NLP)

In [11]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\muyan\AppData\Roaming\nltk_data...


True

In [14]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize stopwords and lemmatizer for efficiency
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define a function for cleaning the text
def clean_text(text):
    if not isinstance(text, str):
        return ""  # Return an empty string instead of None to avoid errors

    # Convert to lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r'[^\w\s\d]', '', text)

    # Tokenize and remove stop words
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Rejoin the words back together
    text = ' '.join(words)

    return text

# Apply the clean_text function to the 'tags' column
new_data['tags_clean'] = new_data['tags'].apply(clean_text)

# Verify the new column
print(new_data[['tags', 'tags_clean']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muyan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\muyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                                tags  \
0  Framed in the 1940s for the double murder of h...   
1  Raj is a rich, carefree, happy-go-lucky second...   
2  Spanning the years 1945 to 1955, a chronicle o...   
3  The true story of how businessman Oskar Schind...   
4  In the continuing saga of the Corleone crime f...   

                                          tags_clean  
0  framed 1940s double murder wife lover upstandi...  
1  raj rich carefree happygolucky second generati...  
2  spanning year 1945 1955 chronicle fictional it...  
3  true story businessman oskar schindler saved t...  
4  continuing saga corleone crime family young vi...  


##  Count Vectorization

In [15]:
# Apply CountVectorizer to the 'tags_clean' column
cv = CountVectorizer(max_features=10000, stop_words='english')

# Transform the text data
vector = cv.fit_transform(new_data['tags_clean'].values.astype('U')).toarray()

# Show the shape of the vectorized data
print(vector.shape)


(10000, 10000)


## Cosine Similarity Calculation

In [16]:
# Calculate cosine similarity
similarity = cosine_similarity(vector)

# Verify the similarity array
print(similarity)


[[1.         0.03175003 0.02993422 ... 0.0862796  0.09274778 0.03745029]
 [0.03175003 1.         0.05892557 ... 0.         0.         0.        ]
 [0.02993422 0.05892557 1.         ... 0.         0.04303315 0.0347524 ]
 ...
 [0.0862796  0.         0.         ... 1.         0.         0.        ]
 [0.09274778 0.         0.04303315 ... 0.         1.         0.        ]
 [0.03745029 0.         0.0347524  ... 0.         0.         1.        ]]


## Movie Recommendation Function

In [17]:
# Define the recommend function
def recommend(movie_title):
    index = new_data[new_data['title'] == movie_title].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector: vector[1])
    for i in distance[1:6]:  # Skipping the first result as it's the movie itself
        print(new_data.iloc[i[0]]['title'])

# Test the recommendation system with a movie title
recommend("Iron Man")


Iron Man 2
Banana
The New Mutants
Time Trap
The Disappearance of Alice Creed


## Save the Data and Similarity Matrices

In [19]:
# Save the new_data and similarity matrices to disk
pickle.dump(new_data, open('movies_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))


## Load and Test Saved Data

In [20]:
# Load the saved data
movies_list = pickle.load(open('movies_list.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

# Verify the loaded data
print(movies_list.head())


      id                        title  \
0    278     The Shawshank Redemption   
1  19404  Dilwale Dulhania Le Jayenge   
2    238                The Godfather   
3    424             Schindler's List   
4    240       The Godfather: Part II   

                                                tags  \
0  Framed in the 1940s for the double murder of h...   
1  Raj is a rich, carefree, happy-go-lucky second...   
2  Spanning the years 1945 to 1955, a chronicle o...   
3  The true story of how businessman Oskar Schind...   
4  In the continuing saga of the Corleone crime f...   

                                          tags_clean  
0  framed 1940s double murder wife lover upstandi...  
1  raj rich carefree happygolucky second generati...  
2  spanning year 1945 1955 chronicle fictional it...  
3  true story businessman oskar schindler saved t...  
4  continuing saga corleone crime family young vi...  


## Download Files from Jupyter (Optional)

In [25]:
# Save to a file (if running on Jupyter)

import shutil

# Save in the current working directory
shutil.copy('similarity.pkl', 'similarity_copy.pkl')

# OR Save to a specific folder (e.g., Downloads)
import os
destination = os.path.join(os.getcwd(), 'similarity_copy.pkl')  # Save in the current directory
shutil.copy('similarity.pkl', destination)

print(f"File saved to: {destination}")


File saved to: C:\Users\muyan\OneDrive - UBC\School\DS\Machine learning\Movie Recommendation System\similarity_copy.pkl
